# Global Health Medical Center -  Bone Fracture Image Classification with PyTorch

## Business Scenario

In the rapidly evolving field of medical diagnostics, timely and accurate detection of bone fractures can significantly impact patient outcomes and healthcare efficiency. Emergency rooms and clinics around the world are often overwhelmed, leading to delayed diagnoses and increased patient suffering. 

Imagine a scenario where a state-of-the-art AI system assists medical professionals by providing rapid, reliable, and precise identification of bone fractures from X-ray images. This technology can help reduce diagnostic errors, expedite treatment decisions, and ultimately save lives. By leveraging advanced machine learning techniques and powerful neural networks, this project aims to develop an automated solution for bone fracture detection.

This bone fracture classification system can be integrated into existing hospital workflows, assisting radiologists in their decision-making process. It can serve as a second opinion, catching fractures that might be missed during a busy shift or by less experienced practitioners. Additionally, in rural or under-resourced settings, where access to expert radiologists is limited, this AI system can provide critical support, ensuring that patients receive the care they need, when they need it.

By leveraging advanced machine learning techniques and a sophisticated convolutional neural network (CNN) architecture, this project seeks to develop a robust and reliable model for bone fracture detection. This AI system will provide significant support in:
- **Reducing diagnostic errors**: Acting as a second pair of eyes for radiologists, the system helps catch fractures that might be missed during hectic shifts or by less experienced practitioners.
- **Expediting treatment decisions**: Rapid and accurate fracture detection enables quicker treatment initiation, improving patient outcomes.
- **Enhancing healthcare accessibility**: In rural or under-resourced settings, where access to expert radiologists is limited, the AI system can provide essential diagnostic support.

Welcome to the future of medical diagnostics, where technology and healthcare unite to enhance patient care and streamline medical processes. Let's dive into the development of this groundbreaking bone fracture image classification model.


In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader , Dataset
from torchvision import datasets, transforms, models
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import deeplake
import time

## Data Preparation

We'll prepare the dataset with data augmentation for the training set and normalization for all sets.

In [2]:
# Data transformations with augmentation

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((224, 224)),  # Resize to a fixed size
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(10),  # Random rotation
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

val_test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


# Load the data
mura_train = deeplake.load("hub://activeloop/mura-train")
mura_val = deeplake.load("hub://activeloop/mura-val")

test_dataset = datasets.ImageFolder(root='bonedata/test', transform=val_test_transform)




train_loader = mura_train.pytorch(num_workers = 8, 
                                shuffle = True, 
                                transform = {'images': train_transform, 'study_type': None}, 
                                batch_size = 64, 
                                decode_method = {'images': 'pil'})


val_loader = mura_val.pytorch(num_workers = 8, 
                                shuffle = False, 
                                transform = {'images': val_test_transform, 'study_type': None}, 
                                batch_size = 64, 
                                decode_method = {'images': 'pil'})

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/mura-train



|

hub://activeloop/mura-train loaded successfully.



Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/mura-val



-

hub://activeloop/mura-val loaded successfully.



## Model Definition

We'll use a pre-trained ResNet model and fine-tune it for our specific task.


In [3]:
model = models.resnet18(weights = models.ResNet18_Weights.DEFAULT)

# Modify the first convolutional layer to accept single-channel input
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Modify the final layer to match the number of classes (binary classification in this case)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


## Loss Function and Optimizer

We'll use the Cross Entropy Loss and the Adam optimizer. A learning rate scheduler will also be used to adjust the learning rate during training.


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

## Training the Model

We'll train the model for a set number of epochs, evaluate it on the validation set, and adjust the learning rate using the scheduler.

let's first define a function for training the model

In [5]:
def model_train(model, optimizer, train_loader, val_loader, criterion, device):
    
    model.train()
    train_loss = 0
    train_correct = 0

    for i, data in enumerate(train_loader):
        
            inputs = data['images']
            labels = torch.squeeze(data['study_type'])
            
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
  

    train_loss /= len(train_loader)
    train_accuracy = train_correct / len(train_loader.dataset)

    # Evaluate on validation set
    model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for j, val_data in enumerate(val_loader):
         
                val_inputs = val_data['images']
                val_labels = torch.squeeze(val_data['study_type'])

                val_inputs = val_inputs.to(device)
                val_labels = val_labels.to(device)
                val_outputs  = model(val_inputs.float())
                loss = criterion(val_outputs, val_labels)
                val_loss += loss.item()
                _, predicted = torch.max(val_outputs, 1)
                val_correct += (predicted == val_labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = val_correct / len(val_loader.dataset)
    



    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
          f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
    return train_accuracy, val_accuracy

now let's train the model

In [6]:
num_epochs = 25
training_start_time = time.time()

#variables for implementing early stopping
patience = 5 # this variable is to apply early stopping technique
best_val_accuracy = 0.0
epochs_without_improvement = 0

for epoch in range(num_epochs):
    print(f"------------------ Training Epoch {epoch+1} /{num_epochs} ------------------")
    train_accuracy, val_accuracy = model_train(model, optimizer, train_loader, val_loader, criterion, device)
    
    # Adjust the learning rate
    scheduler.step()
    
    #If statement for early stopping mechanism
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print("Early stopping triggered")
        break
    
    
training_end_time = time.time()

# Calculate the total training time in hours
total_training_time = (training_end_time - training_start_time) /3600
print(f"Total training time: {total_training_time:.2f} hours")

------------------ Training Epoch 1 /25 ------------------


Please wait, filling up the shuffle buffer with samples.: 100%|██████████████████▉| 1.91G/1.91G [05:01<00:00, 6.78MB/s]


Shuffle buffer filling is complete.
Epoch 1/25, Train Loss: 0.6214, Train Accuracy: 0.6628, Validation Loss: 0.6584, Validation Accuracy: 0.6333
------------------ Training Epoch 2 /25 ------------------


Please wait, filling up the shuffle buffer with samples.: 100%|██████████████████▉| 1.91G/1.91G [05:00<00:00, 6.81MB/s]


Shuffle buffer filling is complete.
Epoch 2/25, Train Loss: 0.5587, Train Accuracy: 0.7141, Validation Loss: 0.5913, Validation Accuracy: 0.7018
------------------ Training Epoch 3 /25 ------------------


...

Shuffle buffer filling is complete.
Epoch 18/25, Train Loss: 0.3772, Train Accuracy: 0.8381, Validation Loss: 0.4898, Validation Accuracy: 0.7881
------------------ Training Epoch 19 /25 ------------------


Please wait, filling up the shuffle buffer with samples.: 100%|██████████████████▉| 1.91G/1.91G [05:10<00:00, 6.60MB/s]


Shuffle buffer filling is complete.
Epoch 19/25, Train Loss: 0.3777, Train Accuracy: 0.8371, Validation Loss: 0.4926, Validation Accuracy: 0.7837
------------------ Training Epoch 20 /25 ------------------


Please wait, filling up the shuffle buffer with samples.: 100%|██████████████████▉| 1.91G/1.91G [04:52<00:00, 6.99MB/s]


Shuffle buffer filling is complete.
Epoch 20/25, Train Loss: 0.3771, Train Accuracy: 0.8392, Validation Loss: 0.4860, Validation Accuracy: 0.7908
Early stopping triggered
Total training time: 6.72 hours


## Conclusion

In this notebook, we built an advanced image classification model using a pre-trained ResNet and fine-tuned it to detect bone fractures. We used data augmentation, a learning rate scheduler, and evaluated the model on separate training, validation, and test sets to ensure robust performance. We also deployed the model on hugging face (deployment code is in seperate notebook)  to test individual images for fracture predictions, making this project a comprehensive and practical tool for bone fracture detection.
